# GRT Development: routine and constants

In [1]:
import spiceypy as spy
from matplotlib import pyplot as plt
import pandas as pd
from time import time,strftime
import numpy as np
from copy import deepcopy
%matplotlib nbagg
TIME=time()
BDIR=f"./"
DATADIR=f"{BDIR}/data/"
KERDIR=f"{BDIR}/util/kernels/"

## Constants

In [2]:
#Numerical
DEG=np.pi/180
RAD=1/DEG

#Phsyical
ELEV=80.0 #km, Reference elevation
AU=1.4959787070000000e8 #km, value assumed in DE430
RMOON=3.84e5 #km

#Source: DE421
#https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/a_old_versions/de421_announcement.pdf
MUSUN=132712440040.944000 #km^3/s^2
MUEARTH=398600.436233 #km^3/s^2
MUMOON=4902.800076 #km^3/s^2
MUEARTHMOON=403503.236310 #km^3/s^2
RHILLEARTH=AU*(MUEARTH/(3*MUSUN))**(1./3)
RHILLMOON=RMOON*(MUMOON/(3*MUEARTH))**(1./3)

#Time constants
DAY=86400
YEAR=365.24*DAY

#Body reference frames

#Kernels
kernels=[
    #Time
    "naif0012.tls",
    #Object properties
    "pck00010.tpc",
    #Planetary positions
    #"de421.bsp",
    "de430.bsp",
    #Earth reference system
    #"earth_720101_070426.bpc","earth_070425_370426_predict.bpc","earth_latest_high_prec_20190910.bpc"
    "earth_latest_high_prec_20161205.bpc",
    "moon_pa_de421_1900-2050.bpc",
    "moon_080317.tf"
    ]
for kernel in kernels:spy.furnsh(f"{KERDIR}/{kernel}")

## Routines

In [3]:
def medHistogram(data,**args):
    h,x=np.histogram(data,**args)
    xm=(x[1:]+x[:-1])/2
    return h,xm

def elTime(verbose=True):
    global TIME
    t=time()
    dt=t-TIME
    if verbose:print("%.3f s, %.3f min, %g h"%(dt,dt/60.0,dt/3600.0))
    TIME=t

In [4]:
def str2t(date):
    et=spy.str2et(date)
    dt=spy.deltet(et,"ET")
    t=et-dt
    return t
spy.str2t=str2t

class Angle(object):
    """
    Routines related to angles
    """
    def dms(value):
        """
        Parameters:
            dec: Angle in decimal, float, degree
        Return:
            dms: Angle in dms, tuple/list/array(4), (sign,deg,min,sec)
        """
        sgn=np.sign(value)
        val=np.abs(value)
        deg=np.floor(val)
        rem=(val-deg)*60
        min=np.floor(rem)
        sec=(rem-min)*60
        return (sgn,deg,min,sec)
    def dec(dms):
        """
        Parameters:
            dms: Angle in dms, tuple/list/array(4), (sign,deg,min,sec)
        Return:
            dec: Angle in decimal, float, degree
        """
        return dms[0]*(dms[1]+dms[2]/60.0+dms[3]/3600.0)

#Testing
import unittest
class Test(unittest.TestCase):

    def test_str2t(self):
        t=str2t("2000 JAN 01 12:00:00")
        self.assertAlmostEqual(t,0,5)

    def test_dms(self):
        dms=Angle.dms(293.231241241)
        self.assertEqual(np.isclose(dms,
                                    (1.0,293.0,13.0,52.46846760007429),
                                    rtol=1e-5).tolist(),
                         [True]*4)

    def test_dms_negative(self):
        dms=Angle.dms(-293.231241241)
        self.assertEqual(np.isclose(dms,
                                    (-1.0,293.0,13.0,52.46846760007429),
                                    rtol=1e-5).tolist(),
                         [True]*4)
        
    def test_dec(self):
        dec=Angle.dec((1,5,40,3.4567))
        self.assertAlmostEqual(dec,5.66762686,5)

    def test_dec_negative(self):
        dec=Angle.dec((-1,5,40,3.4567))
        self.assertAlmostEqual(dec,-5.66762686,5)
        
        
if __name__=='__main__':
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.435s

OK


In [5]:
def zappalaDistance(E1,E2):
    """                                                                                                                                                                              
    Zappala (1990), Nervorny & Vokrouhlicky (2006)
    am=(a+at)/2
    d2c=1/np.sqrt(am)*(ka*((at-a)/am)**2+ke*(et-e)**2+ki*(sinit-np.sin(i*DEG))**2+kO*(Omega-Ot)**2+kw*(omega-ot)**2)
    Parameters:
        E1: Elements 1, np.array(5), [q(UL),e,i(rad),W(rad),w(rad)]
        E2: Elements 1, np.array(5), [q(UL),e,i(rad),W(rad),w(rad)]
    Return:
        DZ: Zappala distance (when [q]=AU, DZ<0.1 is a good match), float
    """
    #Coefficients
    ka=5./4
    ke=ki=2
    kw=kW=1e-4
    #Elements
    q1,e1,i1,W1,w1=E1[:5]
    q2,e2,i2,W2,w2=E2[:5]
    #Derived elements
    sini1=np.sin(i1)
    sini2=np.sin(i2)
    a1=np.abs(q1/(1-e1))
    a2=np.abs(q2/(1-e2))
    am=(a1+a2)/2
    anm=1/np.sqrt(np.abs(am))
    varpi1=W1+w1
    varpi2=W2+w2
    #Zappala metric (Zuluaga & Sucerquia, 2018)
    DZ=anm*(ka*(a1-am)**2/am**2+ke*(e1-e2)**2+ki*(sini1-sini2)**2+kW*(W1-W2)**2+kw*(varpi1-varpi2)**2)**0.5

    return DZ

#Testing
import unittest
class Test(unittest.TestCase):
    def test_Dz(self):
        E1=np.array([1.31435521e-01,7.68157345e-01,1.36782144e+02*DEG,2.80351080e+02*DEG,3.57588822e+02*DEG])
        E2=np.array([1.48116716e-01,7.39354641e-01,1.39995394e+02*DEG,2.80805752e+02*DEG,3.56066191e+02*DEG])
        DZ=zappalaDistance(E1,E2)
        self.assertAlmostEqual(DZ,0.0954985,5)
        
if __name__=='__main__':
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [61]:
"""
Changes log:
- 27/06/2019: Test in the case of Moon GRT
- 27/06/2019: Rotational velocity is positive
"""
class Observer(object):
    """
    lon: longitude, float, radians
    lat: latitude, float, radians
    alt: elevation over reference ellipsoid, float, km
    objid: SPICE Object string (eg. EARTH), string
    """
    _RF=dict(
        EARTH="ITRF93",
        MOON="IAU_MOON" #Others: MOON_PA, MOON_ME
    )
    _PROT=dict(EARTH=DAY,MOON=27.321661*DAY)
    
    def __init__(self,lon,lat,alt,objid):
        self.lon=lon
        self.lat=lat
        self.alt=alt
        self.objid=objid
        radii=spy.bodvrd(self.objid,"RADII",3)[1]
        self._RE=radii[0]
        self._RP=radii[2]
        self._FE=(self._RE-self._RP)/self._RE
        self._positionBody=spy.georec(lon,lat,alt,self._RE,self._FE) 
        self._rho=((self._positionBody[:2]**2).sum())**0.5 #Distance to Earth's axis
        self._velocityLocal=np.array([0,+2*np.pi*self._rho/self._PROT[self.objid],0]) 
        self._calcMatrices()
        self._velocityBody=spy.mxv(self._loc2bod,self._velocityLocal)
    
    def _calcMatrices(self):
        uz=spy.surfnm(self._RE,self._RE,self._RP,self._positionBody)
        uy=spy.ucrss(np.array([0,0,1]),uz)
        ux=spy.ucrss(uz,uy)
        self._loc2bod=np.array(np.vstack((ux,uy,uz)).transpose().tolist())
        self._bod2loc=np.linalg.inv(self._loc2bod)
        self._bod2ecl=None
        self._ecl2bod=None
        
    def updateOrientationPosition(self,et):
        self._et=et
        self._bod2ecl=spy.pxform(self._RF[self.objid],"ECLIPJ2000",et)
        self._ecl2bod=np.linalg.inv(self._bod2ecl)
        self._stateBody=spy.spkezr(self.objid,et,"ECLIPJ2000","NONE","SSB")[0]
        self._positionEcl=spy.mxv(self._bod2ecl,self._positionBody)
        self._positionEcl=self._positionEcl+self._stateBody[:3]
        self._velocityEcl=spy.mxv(self._bod2ecl,self._velocityBody)
        
    def _checkOrientation(self):
        if self._ecl2bod is None:
            raise AssertionError("You must update orientation, eg. observer.updateOrientation(et)")
        
    def ecl2loc(self,eclon,eclat):
        """
        Parameters:
            eclon: Ecliptic longitude, float, radians
            eclat: Ecliptic latitude, float, radians
        Return:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
        """
        self._checkOrientation()
        ecx,ecy,ecz=spy.latrec(1,eclon,eclat)
        x,y,z=spy.mxv(self._bod2loc,spy.mxv(self._ecl2bod,[ecx,ecy,ecz]))
        r,A,h=spy.reclat([x,y,z])
        A=2*np.pi+A if A<0 else A
        return A,h

    def loc2ecl(self,A,h):
        """
        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
        Return:
            eclon: Ecliptic longitude (0,2pi), float, radians
            eclat: Ecliptic latitude (-pi,pi), float, radians
        """
        self._checkOrientation()
        x,y,z=spy.latrec(1,A,h)
        ecx,ecy,ecz=spy.mxv(self._bod2ecl,spy.mxv(self._loc2bod,[x,y,z]))
        r,eclon,eclat=spy.reclat([ecx,ecy,ecz])
        eclon=2*np.pi+eclon if eclon<0 else eclon
        return eclon,eclat

    def vBod2vloc(self,vBod):
        """
        Parameters:
            vBod: Velocity in the body-fixed system, np.array, km/s
        Return:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (-infty,infty), float, km/s
        """
        vLoc=spy.mxv(self._bod2loc,vBod)
        vloc,A,h=spy.reclat(vLoc)
        A=2*np.pi+A if A<0 else A
        return A,h,vloc
    
    def vloc2vBod(self,A,h,vloc):
        """
        Answer the question: what is the velocity (in the rotating geocentric reference frame) 
        of a particle GOING to A,h at a speed vloc. 
        
        It is VERY important to take into account that if a particle is COMING from A,h, you 
        need to specify its velocity as (A,h,-vloc).

        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (-infty,infty), float, km/s
        Return:
            vBod: Velocity in the body-fixed system, np.array, km/s
        """
        self._checkOrientation()
        vLoc=spy.latrec(vloc,A,h)
        vBod=spy.mxv(self._loc2bod,vLoc)
        return vBod
    
    def vloc2vRot(self,A,h,vloc):
        """
        Answer the question: what is the velocity (in the non-rotating geocentric reference frame) 
        of a particle GOING from A,h at a speed vloc

        It is VERY important to take into account that if a particle is COMING from A,h, you 
        need to specify its velocity as (A,h,-vloc).

        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (-infty,infty), float, km/s
        Return:
            vRot: Velocity in the inertial system, body axes, np.array, km/s
        """
        self._checkOrientation()
        vLoc=spy.latrec(vloc,A,h)+self._velocityLocal
        vRot=spy.mxv(self._loc2bod,vLoc)
        return vRot
        
    def vloc2vEcl(self,A,h,vloc):
        """
        Answer the question: what is the velocity (in the heliocentric reference frame) 
        of a particle GOING from A,h to a speed vloc

        It is VERY importante to take into account that if a particle is COMING from A,h, you 
        need to specify its velocity as (A,h,-vloc) or (360-A,h,vloc).

        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (-infty,infty), float, km/s
        Return:
            vEcl: Velocity in the inertial ecliptic system, np.array, km/s
        """
        vRot=self.vloc2vRot(A,h,vloc)
        vEcl=spy.mxv(self._bod2ecl,vRot)+self._stateBody[3:]
        return vEcl

    def helioState(self,A,h,vloc):
        """
        Answer the question: what is the heliocentric state of a particle COMING from A,h to a speed vloc

        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (-infty,infty), float, km/s
        Return:
            bodyState: State vector with respect to sun, np.array (6), (km,km,km,km/s,km/s,km/s)
        """
        vEcl=self.vloc2vEcl(A,h,vloc)
        state=np.concatenate((self._positionEcl,vEcl))
        return state

    def bodyState(self,A,h,vloc):
        """
        Answer the question: what is the heliocentric state of a particle COMING from A,h to a speed vloc

        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (-infty,infty), float, km/s
        Return:
            bodyState: State vector with respect to geocenter, np.array (6), (km,km,km,km/s,km/s,km/s)
        """
        return self.helioState(A,h,vloc)-self._stateBody

    
    def vecl2vEcl(self,eclon,eclat,vloc):
        """
        Parameters:
            eclon: Ecliptic longitude (0,2 pi), float, radians
            eclat: Ecliptic latitude (-pi,pi), float, radians
            vloc: Impact speed in the local reference frame, (0,infty), float, km/s
        Return:
            vEcl: Velocity in the ecliptic system, np.array, km/s
        """
        self._checkOrientation()
        A,h=self.ecl2loc(eclon,eclat)
        vEcl=self.vloc2vEcl(vloc,A,h)
        return vEcl
    
#Testing
import unittest
class Test(unittest.TestCase):
    
    #Zero-observer
    et=spy.str2et("2000 JAN 01 12:00:00 UTC")
    observer=Observer(0*DEG,0*DEG,0,"EARTH")
    observer.updateOrientationPosition(et)

    #Chelyabinsk impact
    etchely=spy.str2et("02/15/2013 3:20:34 UTC")
    chely=Observer(61.1*DEG,54.8*DEG,23.3,"EARTH")
    chely.updateOrientationPosition(etchely)
    
    #Arbitrary impact
    etarb=4320000.0 #Feb. 20/2000, 12:00:00 TDB
    arb=Observer(-29.9483*DEG,-16.4228*DEG,80.0,"EARTH")
    arb._RP=arb._RE;arb._calcMatrices() #Error in GravRay
    arb.updateOrientationPosition(etarb)
    
    #Moon impact
    etmoon=spy.str2et("2000 JAN 02 11:58:56 UTC")
    moon=Observer(45.6452*DEG,41.1274*DEG,10.0,"MOON")
    moon.updateOrientationPosition(etmoon)

    def test_et(self):
        self.assertAlmostEqual(self.et,64.2,1)
        
    def test_ellipsoid(self):
        self.assertAlmostEqual(self.observer._RE,6378.1366,4)
        self.assertAlmostEqual(self.observer._RP,6356.7519,4)
        self.assertAlmostEqual(self.observer._FE,0.0033528131084554717,7)
    
    def test_position(self):
        self.assertEqual(self.observer._positionBody[0],self.observer._RE)
        
    def test_earthpos(self):
        self.assertEqual(np.isclose(self.arb._stateBody,
                                    np.array([-1.30585661e+08,7.08734759e+07,2.98745730e+04,
                                              -1.48394530e+01,-2.62057098e+01,-1.08440413e-03]),
                                    rtol=1e-5).tolist(),
                         [True]*6)

    def test_velocity_local(self):
        self.assertEqual(np.isclose(self.observer._velocityLocal,
                                    np.array([0,+0.46383118,0]),
                                    rtol=1e-5).tolist(),
                         [True]*3)
        
    def test_vrot(self):
        vrot=self.observer.vloc2vRot(90*DEG,0*DEG,-10.0)
        self.assertEqual(np.isclose(vrot,
                                    np.array([0,-9.5361688,0]),
                                    rtol=1e-5).tolist(),
                         [True]*3)

    def test_ecl2loc_sun(self):
        #Position of the Sun
        eclon=Angle.dec((+1,280,22,21.9))
        eclat=Angle.dec((-1,0,0,2.7))
        A,h=self.observer.ecl2loc(eclon*DEG,eclat*DEG)
        self.assertAlmostEqual(A*RAD,178.07231644928592,5)
        self.assertAlmostEqual(h*RAD,66.95277694576103,5)
    
    def test_ecl2loc_Betelgeuse(self):
        #Betelgeuse
        eclon=Angle.dec((+1,88,45,16.6))
        eclat=Angle.dec((-1,16,1,37.2))
        A,h=self.observer.ecl2loc(eclon*DEG,eclat*DEG)
        self.assertAlmostEqual(A*RAD,57.27518638612843,5)
        self.assertAlmostEqual(h*RAD,-76.20677246845091,5)

    def test_arb_position(self):
        self.assertEqual(np.isclose(self.arb._positionEcl,
                                    np.array([-1.30582607e+08,7.08678075e+07,3.03545799e+04]),
                                    rtol=1e-5).tolist(),
                         [True]*3)

    def test_arb_velocity(self):
        vecl=self.arb.vloc2vEcl(1.6502*DEG,56.981*DEG,-12.207)
        self.assertEqual(np.isclose(vecl,
                                    np.array([-20.37576512,-18.14903005,-7.3141155]),
                                    rtol=1e-4).tolist(),
                         [True]*3)

    def test_loc2ecl_sun(self):
        A=Angle.dec((+1,57,16,30.7))
        h=Angle.dec((-1,76,12,24.4))
        eclon,eclat=self.observer.loc2ecl(A*DEG,h*DEG)
        self.assertAlmostEqual(eclon*RAD,88.75461469860417,5)
        self.assertAlmostEqual(eclat*RAD,-16.027004471139914,5)

    def test_chely_et(self):
        self.assertAlmostEqual(self.etchely,414170501.185,3)
        
    def test_chely_vbod2vloc(self):
        vBod=np.array([12.8,-13.3,-2.4])
        A,h,vloc=self.chely.vBod2vloc(-vBod)
        self.assertAlmostEqual(A*RAD,99.8961127649985,5)
        self.assertAlmostEqual(h*RAD,15.92414245029081,5)

    def test_chely_vloc2vbod(self):
        print("Vel.local = ",self.chely._velocityLocal)
        vRot=self.chely.vloc2vRot(101.1*DEG,+15.9*DEG,-18.6)
        print("Vel.rot = ",vRot)
        print("Pos.body = ",self.chely._positionBody)
        print("Vel.body = ",self.chely._velocityBody)
        print("Pos.Ecl.= ",self.chely._positionEcl-self.chely._stateBody[:3])
        vEcl=self.chely.vloc2vEcl(101.1*DEG,+15.9*DEG,-18.6)
        print("Vel.Ecl.= ",vEcl-self.chely._stateBody[3:])
        vBod=self.chely.vloc2vBod(101.1*DEG,+15.9*DEG,-18.6)
        self.assertEqual(np.isclose(vBod,
                                    np.array([12.8,-13.1,-2.4]),
                                    rtol=1e-1).tolist(),
                         [True]*3)

    def test_mooon_observer(self):

        #State moon
        self.assertEqual(np.isclose(self.moon._stateBody,
                                    np.array([-3.03655722e+07,1.43451403e+08,6.46874070e+04,
                                              -2.89028010e+01,-6.57566772e+00,-3.02020818e-02]),
                                    rtol=1e-5).tolist(),
                         [True]*6)

        #Velocity
        vecl=self.moon.vloc2vEcl(1.6502*DEG,56.981*DEG,-4.466),
        self.assertEqual(np.isclose(vecl[0],
                                    np.array([-28.58775394,  -7.71154668,  -4.33777698]),
                                    rtol=1e-5).tolist(),
                         [True]*3)
        #Conversion
        eclon=25.3157371
        eclat=-1.2593327
        A,h=self.moon.ecl2loc(eclon*DEG,eclat*DEG)
        self.assertEqual(np.isclose([A*RAD,h*RAD],
                                    [255.7181,11.6614],
                                    atol=1e-2).tolist(),
                         [True]*2)
        A=229.2705
        h=28.8062
        eclon,eclat=self.moon.loc2ecl(A*DEG,h*DEG)
        self.assertEqual(np.isclose([eclon*RAD,eclat*RAD],
                                    [55.1580499,-5.0588748],
                                    atol=1e-2).tolist(),
                         [True]*2)

if __name__=='__main__':
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

...............

Vel.local =  [0.         0.26894358 0.        ]
Vel.rot =  [ 12.35266075 -13.38862041  -2.17876404]
Pos.body =  [1787.29410694 3237.67768939 5207.6205527 ]
Vel.body =  [-0.23545057  0.1299757   0.        ]
Pos.Ecl.=  [ -864.51344974 -1225.81779918  6208.56473627]
Vel.Ecl.=  [-15.44602301   8.00608843  -5.82351424]



----------------------------------------------------------------------
Ran 15 tests in 0.035s

OK


In [77]:
"""
Changes log:
- 27/06/2019: Test in the case of Moon GRT
- 27/06/2019: Rotational velocity is positive
"""
GLOBAL_OBS=None
def rayOscelt(lon,lat,alt,objid,A,h,vimp,et,objects=["EARTH"],mus=[MUEARTH],rhill=[1e6],simple=True):
    """
    Compute the orbital elements of a particle impacting on lon, lat, alt at time et
    going towards A, h at speed vimp.
    
    Parameters:
        lon: Longitude, float, degrees
        lat: Latitude, float, degrees
        alt: Altitude, float, km
        A: Azimuth, float, degrees
        h: Elevation, float, degrees
        vimp: Impact speed, float (negative), km/s
        et: Time of impact (TDB), float, seconds
    Options:
        objects:
    """
    #Prepare observer structure
    global GLOBAL_OBS
    if (GLOBAL_OBS is None) or \
            np.isclose([lon,lat,alt],[GLOBAL_OBS.lon*RAD,GLOBAL_OBS.lat*RAD,GLOBAL_OBS.alt],rtol=1e-5).sum()<3:
        obs=Observer(lon*DEG,lat*DEG,alt,objid)
        obs.updateOrientationPosition(et)
        GLOBAL_OBS=deepcopy(obs)
    else:
        obs=GLOBAL_OBS
        if np.isclose(et,GLOBAL_OBS._et,rtol=1e-10)==False:
            obs.updateOrientationPosition(et)
    
    #Simple estimation
    if simple:
        heliostate=obs.helioState(A*DEG,h*DEG,vimp)
        E_sim=spy.oscelt(heliostate,et,MUSUN)
    
    E_pcon=[]

    #State respect to the central body
    state=obs.bodyState(A*DEG,h*DEG,vimp)+obs._stateBody
    for k,objid in enumerate(objects):
        print(objid)
        #State respect to object objid
        bodypos=spy.spkezr(objid,et,"ECLIPJ2000","NONE","SSB")[0]
        state=state-bodypos
        #Get object-centric elements
        rt=spy.vnorm(state[:3])
        q,e,i,Omega,omega,Mo,eto,mu=spy.oscelt(state,et,mus[k])
        a=q/(1-e)
        n=np.sqrt(mu/np.abs(a)**3)
        etp=eto-Mo/n
        print("Elements:",etp,n,a,q,e,Omega*RAD,omega*RAD,Mo*RAD,eto,mu,rhill[k])
        #hill
        fd=np.arccos((q*(1+e)/rhill[k]-1)/e)
        #See: https://space.stackexchange.com/questions/27602/what-is-hyperbolic-eccentric-anomaly-f
        Hd=2*np.arctanh(np.sqrt((e-1)/(e+1))*np.tan(fd/2))
        Md=e*np.sinh(Hd)-Hd
        deltat=Md/n
        print("Hyperbolic:",fd,Hd,Md,deltat)
        #Body position
        bodypos=spy.spkezr(objid,etp-deltat,"ECLIPJ2000","NONE","SSB")[0]
        print(bodypos)
        #Heliocentric conic:
        bodyStatet=spy.conics([q,e,i,Omega,omega,Mo,eto,mus[k]],etp-deltat)
        print("BodyStatet = ",bodyStatet)
        hillstate=bodyStatet+bodypos
        #Final orbit
        E_pcon+=[spy.oscelt(hillstate,etp-deltat,MUSUN)]
        #Next state
        et=etp-deltat
        state=hillstate
        print("State at Hill (Heliocentric) = ",state)
    print(E_pcon[-1])
    if simple:
        return E_sim,E_pcon 
    else:
        return E_pcon 

#Testing
import unittest
class Test(unittest.TestCase):

    def test_chely_elements(self):
        et=spy.str2t("02/15/2013 03:20:34 UTC")
        Esim,Epcon=rayOscelt(61.1,54.8,23.3,"EARTH",101.1,15.9,-18.6,et,rhill=[1496558.5265331788])
        Epcon=Epcon[0]
        Epcon[0]/=AU;Epcon[2:5]*=RAD
        q,e,i,W,w=Epcon[:5]
        """
        self.assertEqual(np.isclose([q,e,i,W,w],
                                    [0.78,0.63,4.87,326.57,117.87],
                                     atol=1e-1).tolist(),
                         [True]*5)
        """
        
    def test_moon_elements(self):
        et=spy.str2t("2000 JAN 02 12:00:00 UTC")
        objects=["MOON","EARTH"]
        Esim,Epcon=rayOscelt(45.6452,41.1274,10.0,"MOON",1.6502,56.981,-4.466,et,
                             objects=objects,mus=[MUMOON,MUEARTH],rhill=[61460.05496131042,1496558.5265331788])
        for Ep in Esim,Epcon[0],Epcon[1]:
            Ep[0]/=AU;Ep[2:5]*=RAD

    """
        #Simple
        self.assertEqual(np.isclose(Esim[:5],
                                    np.array([9.16271886e-01,5.53465016e-02,8.34553546e+00,2.82127231e+02,2.83890008e+02]),
                                    atol=1e-5).tolist(),
                         [True]*5)
        self.assertEqual(np.isclose(Epcon[0][:5],
                                    np.array([9.20291177e-01,4.62642311e-02,7.21104714e+00,2.82157775e+02,2.92223420e+02]),
                                    atol=1e-5).tolist(),
                         [True]*5)
        self.assertEqual(np.isclose(Epcon[1][:5],
                                    np.array([9.28333195e-01,4.63308215e-02,6.87468648e+00,2.82227367e+02,2.80901341e+02]),
                                    atol=1e-5).tolist(),
                         [True]*5)
    """
if __name__=='__main__':
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

..

EARTH
Elements: 414170552.05959153 0.0077318558435551235 -1882.1609458763605 6077.673868487962 4.229093602119192 336.8993892375738 112.91422321469491 -52.30071871997865 414170434.0 398600.436233 1496558.5265331788
Hyperbolic: 1.8043497342394161 5.930912182975158 790.1856733717754 102198.70744621208
[-1.21510535e+08  8.38776016e+07 -9.87910669e+03 -1.74608385e+01
 -2.45938146e+01  1.32488512e-03]
BodyStatet =  [ 1.33514266e+06 -6.31599248e+05  2.41171931e+05 -1.30172897e+01
  6.13849933e+00 -2.27591023e+00]
State at Hill (Heliocentric) =  [-1.20175392e+08  8.32460023e+07  2.31292825e+05 -3.04781282e+01
 -1.84553153e+01 -2.27458535e+00]
[1.10490149e+08 5.49669223e-01 7.05389182e-02 5.69976633e+00
 1.86511804e+00 3.72176022e-01 4.14068353e+08 1.32712440e+11]
MOON
Elements: 86749.98971828663 0.011067966351531643 -342.06043311594016 832.0611062542912 3.432497376778644 276.08922903415726 209.23268012922347 -221.9451957819132 86400.00000000006 4902.800076 61460.05496131042
Hyperbolic: 1.84819


----------------------------------------------------------------------
Ran 2 tests in 0.023s

OK


In [ ]:
n=np.sqrt(mu/a**3)
tau=M*DEG/n
sinPhi=e
cos2Phi=1-sinPhi**2
E,cosE,sinE=Ecs(M*DEG,e)
r=a*(1-e*cosE)
miu=cosE/cos2Phi
l=-(sinPhi+cosE)/cos2Phi
m=a**1.5*sinE*(l*sinPhi+2/cos2Phi)
lamb=(a**(0.5)/(r**1))*(1/cos2Phi)*sinE*(r**2-a*r-a**2*cos2Phi)

#d/da: 
#d/de: PARTIALLY CHECKED
l*x+m*vx,l*y+m*vy,l*z+m*vz,lamb*x+miu*vx
#d/di: CHECKED
z*sinW,-z*cosW,-x*sinW+y*cosW,vz*sinW,-vz*cosW,-vx*sinW+vy*cosW
#d/dw: CHECKED
-y*cosi-z*sini*cosW,x*cosi-z*sini*sinW,sini*(x*cosW+y*sinW),\
-vy*cosi-vz*sini*cosW,vx*cosi-vz*sini*sinW,sini*(vx*cosW+vy*sinW)
#d/dW: CHECKED
-y,x,0,-vy,vx,0
#d/dM: CHECKED
a**1.5*vx,a**1.5*vy,a**1.5*vz,-a**1.5/r**3*x,-a**1.5/r**3*y,-a**1.5/r**3*z

In [ ]:
"""
final double[][] jacobian = new double[6][6];

        final double a = this.parameters.getA();
        final double e = this.parameters.getE();
        final double i = this.parameters.getI();
        final double pa = this.parameters.getPerigeeArgument();

        // compute various intermediate parameters
        final PVCoordinates pvc = this.getPVCoordinates();
        final Vector3D position = pvc.getPosition();
        final Vector3D velocity = pvc.getVelocity();
        final Vector3D momentum = pvc.getMomentum();
        final double v2 = velocity.getNormSq();
        final double r2 = position.getNormSq();
        final double r = MathLib.sqrt(r2);
        final double r3 = r * r2;

        final double px = position.getX();
        final double py = position.getY();
        final double pz = position.getZ();
        final double vx = velocity.getX();
        final double vy = velocity.getY();
        final double vz = velocity.getZ();
        final double mx = momentum.getX();
        final double my = momentum.getY();
        final double mz = momentum.getZ();

        final double mu = this.getMu();
        final double sqrtMuA = MathLib.sqrt(a * mu);
        final double sqrtAoMu = MathLib.sqrt(a / mu);
        final double a2 = a * a;
        final double twoA = 2 * a;
        final double rOnA = r / a;

        final double oMe2 = 1 - e * e;
        final double epsilon = MathLib.sqrt(MathLib.max(0.0, oMe2));
        final double sqrtRec = 1 / epsilon;

        final double cosI = MathLib.cos(i);
        final double sinI = MathLib.sin(i);
        final double cosPA = MathLib.cos(pa);
        final double sinPA = MathLib.sin(pa);

        final double pv = Vector3D.dotProduct(position, velocity);
        final double cosE = (a - r) / (a * e);
        final double sinE = pv / (e * sqrtMuA);

        // da
        final Vector3D vectorAR = new Vector3D(2 * a2 / r3, position);
        final Vector3D vectorARDot = velocity.scalarMultiply(2 * a2 / mu);
        fillHalfRow(1, vectorAR, jacobian[0], 0);
        fillHalfRow(1, vectorARDot, jacobian[0], 3);

        // de
        final double factorER3 = pv / twoA;
	System.out.println("factorER3 ="+factorER3);
        final Vector3D vectorER = new Vector3D(cosE * v2 / (r * mu), position, sinE / sqrtMuA,velocity,-factorER3 * sinE / sqrtMuA, vectorAR);
	System.out.println("vectorER ="+vectorER);
        final Vector3D vectorERDot = new Vector3D(sinE / sqrtMuA, position, cosE * 2 * r / mu, velocity, -factorER3 * sinE / sqrtMuA, vectorARDot);
        fillHalfRow(1, vectorER, jacobian[1], 0);
	System.out.println("vectorERDot ="+vectorERDot);
        fillHalfRow(1, vectorERDot, jacobian[1], 3);

        // dE / dr (Eccentric anomaly)
        final double coefE = cosE / (e * sqrtMuA);
        final Vector3D vectorEAnR = new Vector3D(-sinE * v2 / (e * r * mu), position, coefE,
            velocity, -factorER3 * coefE, vectorAR);

        // dE / drDot
        final Vector3D vectorEAnRDot = new Vector3D(-sinE * 2 * r / (e * mu), velocity, coefE,
            position, -factorER3 * coefE, vectorARDot);

        // precomputing some more factors
        final double s1 = -sinE * pz / r - cosE * vz * sqrtAoMu;
        final double s2 = -cosE * pz / r3;
        final double s3 = -sinE * vz / (2 * sqrtMuA);
        final double t1 = sqrtRec * (cosE * pz / r - sinE * vz * sqrtAoMu);
        final double t2 = sqrtRec * (-sinE * pz / r3);
        final double t3 = sqrtRec * (cosE - e) * vz / (2 * sqrtMuA);
        final double t4 = sqrtRec * (e * sinI * cosPA * sqrtRec - vz * sqrtAoMu);
        final Vector3D s = new Vector3D(cosE / r, Vector3D.PLUS_K, s1, vectorEAnR, s2, position,
            s3, vectorAR);
        final Vector3D sDot = new Vector3D(-sinE * sqrtAoMu, Vector3D.PLUS_K, s1, vectorEAnRDot,
            s3, vectorARDot);
        final Vector3D t = new Vector3D(sqrtRec * sinE / r, Vector3D.PLUS_K).add(new Vector3D(t1,
            vectorEAnR, t2, position, t3, vectorAR, t4, vectorER));
        final Vector3D tDot = new Vector3D(sqrtRec * (cosE - e) * sqrtAoMu, Vector3D.PLUS_K, t1,
            vectorEAnRDot, t3, vectorARDot, t4, vectorERDot);

        // di
        final double factorI1 = -sinI * sqrtRec / sqrtMuA;
        final double i1 = factorI1;
        final double i2 = -factorI1 * mz / twoA;
        final double i3 = factorI1 * mz * e / oMe2;
        final double i4 = cosI * sinPA;
        final double i5 = cosI * cosPA;
        fillHalfRow(i1, new Vector3D(vy, -vx, 0), i2, vectorAR, i3, vectorER, i4, s, i5, t,
            jacobian[2], 0);
        fillHalfRow(i1, new Vector3D(-py, px, 0), i2, vectorARDot, i3, vectorERDot, i4, sDot, i5,
            tDot, jacobian[2], 3);

        // dpa
        fillHalfRow(cosPA / sinI, s, -sinPA / sinI, t, jacobian[3], 0);
        fillHalfRow(cosPA / sinI, sDot, -sinPA / sinI, tDot, jacobian[3], 3);

        // dRaan
        final double factorRaanR = 1 / (mu * a * oMe2 * sinI * sinI);
        fillHalfRow(-factorRaanR * my, new Vector3D(0, vz, -vy), factorRaanR * mx, new Vector3D(
            -vz, 0, vx), jacobian[4], 0);
        fillHalfRow(-factorRaanR * my, new Vector3D(0, -pz, py), factorRaanR * mx, new Vector3D(pz,
            0, -px), jacobian[4], 3);

        // dM
        fillHalfRow(rOnA, vectorEAnR, -sinE, vectorER, jacobian[5], 0);
        fillHalfRow(rOnA, vectorEAnRDot, -sinE, vectorERDot, jacobian[5], 3);
"""

In [ ]:
def computeJacobinaKepler2Cartesian(a,e,i,w,W,M,mu):
    """
    Compute the Jacobian Matrix of the transformation from classical 
    orbital elements (a,e,i,w,W,M) to cartesian state vector (x,y,z,x',y',z').
    
    
    Parameters:
        a: Semimajor axis (-inf,inf) (a<0 for hyperbola), float, units compatible with mu
        e: Eccentricity (0,inf) (not including e = 1), float, adminesional 
        i: Inclination [0,pi], float, radians
        w: Argument of the periapse [0,2*pi), float, radians
        W: Longitude of the ascending node [0,2*pi), float, radians
        M: Mean anomaly [0,2*pi), float, radians
        mu: Gravitational parameter (G M), float, units compatible with a
    
    Return:
    
        Jc2k = [dx/da,dx/de,dx/di,dx/dw,dx/dW,dx/dM,
                dy/da,dy/de,dy/di,dy/dw,dy/dW,dy/dM,
                dz/da,dz/de,dz/di,dz/dw,dz/dW,dz/dM,
                dx'/da,dx'/de,dx'/di,dx'/dw,dx'/dW,dx'/dM,
                dy'/da,dy'/de',dy'/di,dy'/dw,dy'/dW,dy'/dM,
                dz'/da,dz'/de,dz'/di',dz'/dw,dz'/dW,dz'/dM],

                Numpy 6x6 array, units compatible with mu and a.
    """
    #Conic signature: s = 1 (ellipse), s = -1 (hyperbola)
    if e<1:s=1
    else:s=-1

    #Trigonometric function
    sini=np.sin(i);cosi=np.cos(i)
    sinw=np.sin(w);cosw=np.cos(w)
    sinW=np.sin(W);cosW=np.cos(W)
    
    #Components of the rotation matrix
    A=(cosW*cosw-cosi*sinW*sinw);B=(-cosW*sinw-cosw*cosi*sinW)
    C=(cosw*sinW+sinw*cosi*cosW);D=(-sinw*sinW+cosw*cosi*cosW)
    F=sinw*sini;G=cosw*sini
    
    #Primary auxiliar variables
    q=a*(1-e)
    ab=np.abs(a)
    n=np.sqry(mu/ab**3)
    nu=n*a**2
    eps=np.sqrt(s*(1-e**2))
    
    #Get cartesian coordinates
    x,y,z,vx,vy,vz=spy.conics([q,e,i,W,w,M,0,mu],0)
    r=(x**2+y**2+z**2)**0.5
    nur=nu/r
    
    #Secondary auxiliar variables
    cosE=vy*r/(nu*eps)
    sinE=-s*vx*r/nu
    
    #dX/da
    Ja=np.array([x/a,y/a,z/a,-vx/(2*a),-vy/(2*a),-vz/(2*a)])
    
    #dX/de
    dcosEde=-s*a*sinE**2/r
    dsinEde=a*cosE*sinE/r
    dnurde=(nu*a/r**2)*(cosE-(ab/r)*e*sinE**2)
    depsde=-s*e/eps
    
    drAde=a*(dcosEde-1)
    drBde=ab*(depsde*sinE+eps*dsinEde)
    
    dvAde=-(dnurde*sinE+nur*dsinEde)
    dvBde=(dnurde*eps*cosE+nur*depsde*cosE+nur*eps*dcosEde)
    
    Je=np.array([
        drAde*A+drBde*B,
        drAde*C+drBde*D,
        drAde*F+drBde*G,
        dvAde*A+dvBde*B,
        dvAde*C+dvBde*D,
        dvAde*F+dvBde*G,
    ])
    
    #dX/di
    Ji=np.array([z*sinW,-z*cosW,-x*sinW+y*cosW,vz*sinW,-vz*cosW,-vx*sinW+vy*cosW])
    
    #dX/dw
    Jw=np.array([-y*cosi-z*sini*cosW,x*cosi-z*sini*sinW,sini*(x*cosW+y*sinW),\
        -vy*cosi-vz*sini*cosW,vx*cosi-vz*sini*sinW,sini*(vx*cosW+vy*sinW)])
    
    #dX/dW
    JW=np.array([-y,x,0,-vy,vx,0])
    
    #dX/dM
    JM=ab**1.5*np.array([vx,vy,vz,-x/r**3,-y/r**3,-z/r**3])
    
    return np.array([Ja,Je,Ji,Jw,JW,JM]).transpose()

In [ ]:
lon=25*DEG
lat=53*DEG
alt=100 # km
et=spy.str2et("2000 JAN 01 12:00:00 UTC")
obs=Observer(lon,lat,alt,"EARTH")
obs.updateOrientationPosition(et)
print("SPICE:",obs._positionBody)
print("Analytic:",any_geo2bod(lon,lat,alt,R,f))
A=40.0*DEG
h=20.0*DEG
vimp=-10.0 #km/s
print("SPICE body:",obs.vloc2vBod(A,h,vimp))
vloc=any_loc2vbod(A,h,vimp)
vbod=spy.mxv(obs._loc2bod,vloc)
print("Analytic:",vbod)
vbod=any_vloc2vbod(lon,lat,A,h,vimp)
print("Full analytic:",vbod)
print("SPICE rotating:",obs.vloc2vRot(A,h,vimp))
vrot=any_vloc2vrot(lon,lat,alt,A,h,vimp,R,f)
print("Rotating full analytic:",vrot)

In [ ]:
def any_geo2bod(lon,lat,alt,R=1,f=0):
    a=R
    b=a*(1-f)
    N=a**2/np.sqrt(a**2*np.cos(lat)**2+b**2*np.sin(lat)**2)
    x=(N+alt)*np.cos(lat)*np.cos(lon)
    y=(N+alt)*np.cos(lat)*np.sin(lon)
    z=(b**2*N/a**2+alt)*np.sin(lat)
    return np.array([x,y,z])

def any_vloc2vrot(lon,lat,alt,A,h,v,R=1,f=0):
    a=R
    b=a*(1-f)

    slon=np.sin(lon);clon=np.cos(lon)
    slat=np.sin(lat);clat=np.cos(lat)
    sA=np.sin(A);cA=np.cos(A)
    sh=np.sin(h);ch=np.cos(h)

    N=a**2/np.sqrt(a**2*clat**2+b**2*slat**2)
    x=(N+alt)*clat*clon
    y=(N+alt)*clat*slon
    vr=2*np.pi*np.sqrt(x**2+y**2)/DAY
    fr=vr/v
    
    vx=v*(-ch*cA*slat*clon-(ch*sA+fr)*slon+sh*clat*clon)
    vy=v*(-ch*cA*slat*slon+(ch*sA+fr)*clon+sh*clat*slon)
    vz=v*(ch*cA*clat+sh*slat)
    
    return np.array([vx,vy,vz])

def local2rotating(xloc,R=1,f=0):
    lon,lat,alt,A,h,v=xloc
    r=any_geo2bod(lon,lat,alt,R=R,f=f)
    v=any_vloc2vrot(lon,lat,alt,A,h,v,R=R,f=f)
    return np.concatenate((r,v))

def computeJacobinaLocal2Cartesian(lon,lat,alt,A,h,vimp,R=1,f=0,P=1):
    """
    Compute the Jacobian Matrix of the transformation from 
    local impact conditions (lon,lat,alt,A,h,v) to cartesian state vector (x,y,z,x',y',z') 
    (in the body reference frame).
    
    
    Parameters:
        lon: Geographic longitude (0,2pi), float, radians
        lat: Geographic latitude (0,2pi), float, radians
        alt: Altitude over the ellipsoid (0,inf), float, km
        A: Azimuth (0,2pi), float, radians
        h: Elevation (-pi/2,pi/2), float, radians
        v: Impact speed (-inf,inf), float, km/s (negative if it is impacting)
    
    Return:

        Jc2l = [dx/dlon,dx/dlat,dx/dalt,dx/dA,dx/dh,dx/dv,
                dy/dlon,dy/dlat,dy/dalt,dy/dA,dy/dh,dy/dv,
                dz/dlon,dz/dlat,dz/dalt,dz/dA,dz/dh,dz/dv,
                dx'/dlon,dx'/dlat,dx'/dalt,dx'/dA,dx'/dh,dx'/dv,
                dy'/dlon,dy'/dlat,dy'/dalt,dy'/dA,dy'/dh,dy'/dv,
                dz'/dlon,dz'/dlat,dz'/dalt,dz'/dA,dz'/dh,dz'/dv],

                Numpy 6x6 array.
    """
    
    #Local to rotating
    X=[lon,lat,alt,A,h,vimp]
    x,y,z,vx,vy,vz=local2rotating(X)
    
    #Trignometric
    coslon=np.cos(lon);sinlon=np.sin(lon)
    coslat=np.cos(lat);sinlat=np.sin(lat)
    cosA=np.cos(A);sinA=np.sin(A)
    cosh=np.cos(h);sinh=np.sin(h)
    
    #Auxiliar
    fr=2*np.pi*np.sqrt(x**2+y**2)/(P*vimp)
    a=R;b=a*(1-f)
    N=a**2/np.sqrt(a**2*coslat**2+b**2*sinlat**2)
    n2=(2*np.pi/P)**2
    
    #dX/dlon:
    Jlon=np.array([-y,x,0,-vy,vx,0])
    
    #dX/dlat:
    dxdlat=(a**2-b**2)*coslat*sinlat*N**3/a**4*coslat*coslon-(N+alt)*sinlat*coslon
    dydlat=(a**2-b**2)*coslat*sinlat*N**3/a**4*coslat*sinlon-(N+alt)*sinlat*sinlon
    Jlat=np.array([
        dxdlat,
        dydlat,
        b**2*(a**2-b**2)*coslat*sinlat*N**3/a**6*sinlat+(b**2*N/a**2+alt)*coslat,
        -vimp*cosh*cosA*coslat*coslon-n2*sinlon/(fr*vimp)*(x*dxdlat+y*dydlat)-vimp*sinh*sinlat*coslon,
        -vimp*cosh*cosA*coslat*sinlon+n2*coslon/(fr*vimp)*(x*dxdlat+y*dydlat)-vimp*sinh*sinlat*sinlon,
        vimp*(-cosh*cosA*sinlat+sinh*coslat)
    ])
    
    #dX/dalt:
    Jalt=np.array([
        coslat*coslon,coslat*sinlon,sinlat,
        -n2*sinlon/(fr*vimp)*(x*coslat*coslon+y*coslat*sinlon),
        +n2*coslon/(fr*vimp)*(x*coslat*coslon+y*coslat*sinlon),
        0
    ])
    
    #dX/dA:
    JA=np.array([0,0,0,
        vimp*(cosh*sinA*sinlat*coslon-cosh*cosA*sinlon),
        vimp*(cosh*sinA*sinlat*sinlon+cosh*cosA*coslon),
        -vimp*cosh*sinA*coslat,
       ])
    
    #dX/dh:
    Jh=np.array([0,0,0,
        vimp*(sinh*cosA*sinlat*coslon+sinh*sinA*sinlon+cosh*coslat*coslon),
        vimp*(sinh*cosA*sinlat*sinlon-sinh*sinA*coslon+cosh*coslat*sinlon),
        vimp*(-sinh*cosA*coslat+cosh*sinlat),
        ])

    #dX/dvimp:
    Jv=np.array([0,0,0,vx/vimp+sinlon*fr,vy/vimp-coslon*fr,vz/vimp])
    
    return np.array([Jlon,Jlat,Jalt,JA,Jh,Jv]).transpose()